In [ ]:
# @formatter:off
import random

import numpy as np
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
# @formatter:on
import os

os.chdir('../')
print(os.getcwd())

In [ ]:
#@formatter:off
files = !ls data/*csv.jsonl
#@formatter:on
files = [f for f in files if 'sample' not in f and 'augmented' not in f and 'processed' not in f]
assert len(files) == 8

In [ ]:
#@formatter:off
snippet_files = !ls data/*-google-results-sample.jsonl | grep -v snippets
#@formatter:on

In [ ]:
print(snippet_files)
files

In [ ]:
import json
from tqdm import tqdm

from unidecode import unidecode


def load_json(d):
    try:
        return json.loads(d, strict=False)
    except Exception as e:
        print("FEK")
        print(d)
        print(unidecode(d))
        return json.loads(unidecode(d), strict=False)


def load_jsonl(path: str):
    print("loading", path)
    with open(path) as f:
        return [load_json(d) for d in tqdm(f.readlines())]


snippet_datasets = [load_jsonl(p) for p in snippet_files]
datasets = [load_jsonl(p) for p in files]

In [ ]:
snippet_map = {
    k: files.index(k.replace('-google-results-sample.jsonl', '.csv.jsonl')) for k in snippet_files
}
snippet_map

In [ ]:
def process_snippet(d, claim_ds):
    if d['results'] and d['results'].get('items', None):
        snippets_formatted = '\n'.join(
            t.get('snippet', '').split('...', 1)[-1].replace('\xa0...', '').replace('...', '...\n...').strip() for t in d['results']['items']).strip()
    else:
        snippets_formatted = ''
    # print(snippets_formatted)
    claim = claim_ds[d['id']]['text']
    explanataion = claim_ds[d['id']]['cR_textualRating']
    text_article = claim_ds[d['id']]['text_article']
    return {
        'input_snippet': '\n'.join((claim, snippets_formatted)),
        'input_article': f"{claim}\n{text_article}",
        'target': explanataion
    }


ff_map = {d['id']: d for d in datasets[3]}
processed_fullfact_snippets = [process_snippet(s, ff_map) for s in snippet_datasets[0]]

In [ ]:
from handystuff.loaders import write_jsonl


def write_out_ds(ds, f):
    f = f.replace('data/', 'data/snippets-')
    write_jsonl(ds, f)

write_out_ds(processed_fullfact_snippets, snippet_files[0])
